## Imports

In [2]:
# Object-oriented
from abc import ABC, abstractmethod, abstractproperty

# Math 
import random
import numpy as np
from scipy.spatial import distance

# System
import sys
import io

# Plotting, widgets
import seaborn as sns
from ipywidgets import *
import matplotlib.patches
import matplotlib.pyplot as plt
from IPython.display import clear_output, display
from IPython.display import HTML as html
%matplotlib inline
sns.set(style="ticks")
plt.rcParams["font.family"] = "Arial"
display(HTML('''<style>
            .widget-label {min-width: 20ex !important;}
            .widget-text {width: 30ex;}
            </style>'''))

# Set random seed
RANDOM_SEED = 0

HTML(value='<style>\n            .widget-label {min-width: 20ex !important;}\n            .widget-text {width:…

In [3]:
%%capture

# Importing from other ipynbs that encapsulate classes with low-level design
import ipynb
import importlib

import ipynb.fs.full.Firefly as Firefly
import ipynb.fs.full.World as World

# Each time the low-level code is changed, just run this to reload
importlib.reload(Firefly)
importlib.reload(World)

In [4]:
firefly_world = World.World()

## Widget time

In [5]:
class WidgetCollection:
    def __init__(self, firefly_world):
        self.firefly_world = firefly_world
        
        # Layout style for many widgets
        self.layout = Layout(border='0px solid red', width='350px', height='30px')
        
        # Generates all widgets
        self.generate_collection()
        
    def generate_textbox(self):
        self.ticks = Text(description="Ticks:", layout=Layout(width='200px', height='30px', margin='0px 0px 0px 60px'))
        
    def generate_buttons(self):
        self.setup_button = Button(description="setup/reset")
        self.setup_button.style.button_color = 'lightgreen'
        
        self.go_button = Button(description="go")
        self.go_button.style.button_color = 'lightgreen'
        
    def generate_plots(self):
        self.plot_data = HTML()
        self.plot_simulation = HTML()

    def generate_sliders(self):
        self.num_males_slider = IntSlider(description="Number of fireflies:", value=30, min=1, max=100, layout=self.layout)
        self.num_males_slider.style.handle_color = 'lightgreen'
        
        self.step_size_slider = IntSlider(description="Step size:", value=1, min=1, max=5, layout=self.layout)
        self.step_size_slider.style.handle_color = 'lightgreen'

        self.num_steps_slider = IntSlider(description="Number of ticks:", value=50, min=10, max=300, layout=self.layout)
        self.num_steps_slider.style.handle_color = 'lightgreen'
        
        self.turning_angle_slider = FloatSlider(description="Turning angle width:", value=np.pi/8, min=np.pi/15, max=np.pi/5, layout=self.layout)
        self.turning_angle_slider.style.handle_color = 'lightgreen'
        
        self.flash_min_slider = IntSlider(description="Flashing interval min:", value=5, min=1, max=15, layout=self.layout)
        self.flash_min_slider.style.handle_color = 'lightgreen'
        
        self.flash_max_slider = IntSlider(description="Flashing interval max:", value=30, min=10, max=100, layout=self.layout)
        self.flash_max_slider.style.handle_color = 'lightgreen'
        
    def generate_checkbox(self):
        self.trackon_checkbox = Checkbox(description="Show track", layout=Layout(width='500px', margin='0px 0 -35px 0px'))
    
    def update_widgets(self):
        self.ticks.value = str(self.firefly_world.ticks)
        self.plot_data.value = str(self.firefly_world.plot_data())
        self.plot_simulation.value = str(self.firefly_world.plot_simulation(int(self.ticks.value)))
        self.num_males_slider.value = self.firefly_world.num_males
        self.step_size_slider.value = self.firefly_world.firefly_collection.step_size
        self.num_steps_slider.value = self.firefly_world.num_total_steps
        self.turning_angle_slider.value = self.firefly_world.firefly_collection.turning_angle_distribution
        self.flash_min_slider.value = self.firefly_world.firefly_collection.flash_interval_min
        self.flash_max_slider.value = self.firefly_world.firefly_collection.flash_interval_max
        self.trackon_checkbox.value = self.firefly_world.track_on
        
    def generate_collection(self):
        
        ''' All the above widgets into one with arrangement in boxes. '''
        
        self.generate_textbox()
        self.generate_buttons()
        self.generate_plots()
        self.generate_sliders()
        self.generate_checkbox()
        
        row1 = HBox([self.setup_button, self.go_button, self.trackon_checkbox])
        row2 = HBox([self.num_males_slider, self.step_size_slider])
        row3 = HBox([self.num_steps_slider, self.turning_angle_slider])
        row4 = HBox([self.flash_min_slider, self.flash_max_slider])
        row5 = HBox([self.plot_data], layout=Layout(height="350px", width="500px"))
        self.widgets = HBox([VBox([row1, row2, row3, row4, row5], layout=Layout(width="600px")), 
                        VBox([self.ticks, self.plot_simulation], layout=Layout(width="500px"))])
       
        self.update_widgets()
        
    def __call__(self):
        return self.widgets

In [6]:
widgets_collection = WidgetCollection(firefly_world)
# widgets = widgets_collection()

In [7]:
class SimulationGUI:
    
    def __init__(self, widgets_collection, firefly_world):
        self.widgets_collection = widgets_collection
        self.firefly_world = firefly_world
        
        self.gui = self.run_simulation()
    
    def update(self, args):
        if args["name"] != "value":
            return
        owner = args["owner"]
        name = args["name"]
        value = args["new"]

        if owner == self.widgets_collection.num_males_slider:
            self.firefly_world.num_males = value
        elif owner == self.widgets_collection.step_size_slider:
            self.firefly_world.step_size = value
        elif owner == self.widgets_collection.num_steps_slider:
            self.firefly_world.num_total_steps = value
        elif owner == self.widgets_collection.turning_angle_slider:
            self.firefly_world.turning_angle_distribution = value
        elif owner == self.widgets_collection.flash_min_slider:
            self.firefly_world.flash_interval_min = value
        elif owner == self.widgets_collection.flash_max_slider:
            self.firefly_world.flash_interval_max = value
        elif owner == self.widgets_collection.trackon_checkbox:
            self.firefly_world.track_on = value
        
    def setup(self, widgets):
        self.firefly_world.setup()
        self.widgets_collection.plot_data.value = self.firefly_world.plot_data()
        self.widgets_collection.plot_simulation.value = self.firefly_world.plot_simulation(int(self.widgets_collection.ticks.value))
        self.widgets_collection.ticks.value = str(self.firefly_world.ticks)
        
    def run(self, widgets):
        if self.firefly_world.state == "stopped":
            w = {"plot_data": self.widgets_collection.plot_data, 
                 "ticks": self.widgets_collection.ticks, 
                 "plot_simulation": self.widgets_collection.plot_simulation} 
            self.firefly_world.run(w)
            
    def run_simulation(self):
        self.widgets_collection.num_males_slider.observe(self.update)         
        self.widgets_collection.step_size_slider.observe(self.update)
        self.widgets_collection.num_steps_slider.observe(self.update)
        self.widgets_collection.turning_angle_slider.observe(self.update)
        self.widgets_collection.flash_min_slider.observe(self.update)
        self.widgets_collection.flash_max_slider.observe(self.update)
        self.widgets_collection.trackon_checkbox.observe(self.update)

        self.widgets_collection.setup_button.on_click(self.setup)
        self.widgets_collection.go_button.on_click(self.run)
    
        return self.widgets_collection


In [8]:
GUI = SimulationGUI(widgets_collection, firefly_world)
GUI.gui()

## functionalized

In [ ]:
def update(args):
    if args["name"] != "value":
        return
    owner = args["owner"]
    name = args["name"]
    value = args["new"]
    
    if owner == widgets_collection.num_males_slider:
        firefly_world.num_males = value
    elif owner == widgets_collection.step_size_slider:
        firefly_world.step_size = value
    elif owner == widgets_collection.num_steps_slider:
        firefly_world.num_total_steps = value
    elif owner == widgets_collectionturning_angle_slider:
        firefly_world.turning_angle_distribution = value
    elif owner == widgets_collection.flash_min_slider:
        firefly_world.flash_interval_min = value
    elif owner == widgets_collection.flash_max_slider:
        firefly_world.flash_interval_max = value
    elif owner == widgets_collection.trackon_checkbox:
        firefly_world.track_on = value
        
def setup(widgets):
    firefly_world.setup()
    widgets_collection.plot_data.value = firefly_world.plot_data()
    widgets_collection.plot_simulation.value = firefly_world.plot_simulation(int(widgets_collection.ticks.value))
    widgets_collection.ticks.value = str(firefly_world.ticks)
        
def run(widgets):
    if firefly_world.state == "stopped":
        w = {"plot_data": widgets_collection.plot_data, 
             "ticks": widgets_collection.ticks, 
             "plot_simulation": widgets_collection.plot_simulation} 
        firefly_world.run(w)  

def run_simulation():
    widgets_collection.num_males_slider.observe(update)         
    widgets_collection.step_size_slider.observe(update)
    widgets_collection.num_steps_slider.observe(update)
    widgets_collection.turning_angle_slider.observe(update)
    widgets_collection.flash_min_slider.observe(update)
    widgets_collection.flash_max_slider.observe(update)
    widgets_collection.trackon_checkbox.observe(update)

    widgets_collection.setup_button.on_click(setup)
    widgets_collection.go_button.on_click(run)
    
run_simulation()
widgets_collection()

In [ ]:
class SimulationGUI:
    def __init__(self, firefly_world, widgets_collection):
        self.firefly_world = firefly_world
        self.widgets_collection = widgets_collection
        
        self.run_simulation()
        
    def update(self, args):
        if args["name"] != "value":
            return

        owner = args["owner"]
        name = args["name"]
        value = args["new"]

        if owner == self.widgets_collection.num_males_slider:
            self.firefly_world.num_males = value
        elif owner == self.widgets_collection.step_size_slider:
            self.firefly_world.step_size = value
        elif owner == self.widgets_collection.num_steps_slider:
            self.firefly_world.num_total_steps = value
        elif owner == self.widgets_collection.turning_angle_slider:
            self.firefly_world.turning_angle_distribution = value
        elif owner == self.widgets_collection.flash_min_slider:
            self.firefly_world.flash_interval_min = value
        elif owner == self.widgets_collection.flash_max_slider:
            self.firefly_world.flash_interval_max = value
        elif owner == self.widgets_collection.trackon_checkbox:
            self.firefly_world.track_on = value
        
    def setup(self, widgets):
        self.firefly_world.setup()
        self.widgets_collection.plot_data.value = self.firefly_world.plot_data()
        self.widgets_collection.plot_simulation.value = self.firefly_world.plot_simulation(int(self.widgets_collection.ticks.value))
        self.widgets_collection.ticks.value = str(self.widgets_collection.firefly_world.ticks)

    def run(self, widgets):
        if self.firefly_world.state == "stopped":
            w = {"plot_data": self.widgets_collection.plot_data, 
                 "ticks": self.widgets_collection.ticks, 
                 "plot_simulation": self.widgets_collection.plot_simulation} 
            self.firefly_world.run(self.widgets_collection)  

    def run_simulation(self):
        self.widgets_collection.num_males_slider.observe(self.update)         
        self.widgets_collection.step_size_slider.observe(self.update)
        self.widgets_collection.num_steps_slider.observe(self.update)
        self.widgets_collection.turning_angle_slider.observe(self.update)
        self.widgets_collection.flash_min_slider.observe(self.update)
        self.widgets_collection.flash_max_slider.observe(self.update)
        self.widgets_collection.trackon_checkbox.observe(self.update)

        self.widgets_collection.setup_button.on_click(self.setup)
        self.widgets_collection.go_button.on_click(self.run)

In [ ]:
simulation = SimulationGUI(firefly_world, widgets_collection)
widgets = simulation.widgets_collection()

widgets

## Old version of GUI

In [ ]:
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

layout = Layout(border='0px solid red', width='350px', height='30px',
               margin='0px 0px 0px 0px')

# Text
ticks = Text(description="Ticks:", layout=Layout(border='0px solid red', width='200px', height='30px', margin='0px 0px 0px 60px'))

# Buttons
setup_button = Button(description="setup/reset")
go_button = Button(description="go")

# Plots
plot_data = HTML()
plot_simulation = HTML(layout=Layout(border='0px solid red', margin='0px 0px 0px 0px'))

# Sliders
num_males_slider = IntSlider(description="Number of fireflies:", value=30, min=1, max=100, layout=layout)
step_size_slider = IntSlider(description="Step size:", value=1, min=1, max=5, layout=layout)
num_steps_slider = IntSlider(description="Number of ticks:", value=50, min=10, max=300, layout=layout)
turning_angle_slider = FloatSlider(description="Turning angle width:", value=np.pi/8, min=np.pi/15, max=np.pi/5, layout=layout)
flash_min_slider = IntSlider(description="Flashing interval min:", value=5, min=1, max=15, layout=layout)
flash_max_slider = IntSlider(description="Flashing interval max:", value=30, min=10, max=100, layout=layout)

# Checkbox
trackon_checkbox = Checkbox(description="Show track", layout=Layout(border='0px solid red', width='500px', margin='0px 0 -35px 0px'))

##################################################################

# Color settings
setup_button.style.button_color = 'lightgreen'
go_button.style.button_color = 'lightgreen'
num_males_slider.style.handle_color = 'lightgreen'
step_size_slider.style.handle_color = 'lightgreen'
num_steps_slider.style.handle_color = 'lightgreen'
turning_angle_slider.style.handle_color = 'lightgreen'
flash_min_slider.style.handle_color = 'lightgreen'
flash_max_slider.style.handle_color = 'lightgreen'

##################################################################
# Layout
row1 = HBox([setup_button, go_button, trackon_checkbox])
row2 = HBox([num_males_slider, step_size_slider])
row3 = HBox([num_steps_slider, turning_angle_slider])
row4 = HBox([flash_min_slider, flash_max_slider])
row5 = HBox([plot_data], layout=Layout(border='solid 0px lightgreen', height="350px", width="500px"))
widgets = HBox([VBox([row1, row2, row3, row4, row5], layout=Layout(border='solid 0px black', width="600px")), 
                VBox([ticks, plot_simulation], layout=Layout(border='solid 0px black', width="500px"))])

#################################################################
# Update
ticks.value = str(firefly_world.ticks)
plot_data.value = str(firefly_world.plot_data())
plot_simulation.value = str(firefly_world.plot_simulation(int(ticks.value)))
num_males_slider.value = firefly_world.num_males
step_size_slider.value = firefly_world.firefly_collection.step_size
num_steps_slider.value = firefly_world.num_total_steps
turning_angle_slider.value = firefly_world.firefly_collection.turning_angle_distribution
flash_min_slider.value = firefly_world.firefly_collection.flash_interval_min
flash_max_slider.value = firefly_world.firefly_collection.flash_interval_max
trackon_checkbox.value = firefly_world.track_on

# def update(args):
#     if args["name"] != "value":
#         return
#     owner = args["owner"]
#     name = args["name"]
#     value = args["new"]
    
#     if owner == num_males_slider:
#         firefly_world.num_males = value
#     elif owner == step_size_slider:
#         firefly_world.step_size = value
#     elif owner == num_steps_slider:
#         firefly_world.num_total_steps = value
#     elif owner == turning_angle_slider:
#         firefly_world.turning_angle_distribution = value
#     elif owner == flash_min_slider:
#         firefly_world.flash_interval_min = value
#     elif owner == flash_max_slider:
#         firefly_world.flash_interval_max = value
#     elif owner == trackon_checkbox:
#         firefly_world.track_on = value
        
# def setup(widgets):
#     firefly_world.setup()
#     plot_data.value = firefly_world.plot_data()
#     plot_simulation.value = firefly_world.plot_simulation(int(ticks.value))
#     ticks.value = str(firefly_world.ticks)
        
# def run(widgets):
#     if firefly_world.state == "stopped":
#         w = {"plot_data": plot_data, "ticks": ticks, "plot_simulation": plot_simulation} 
#         firefly_world.run(w)  

# num_males_slider.observe(update)         
# step_size_slider.observe(update)
# num_steps_slider.observe(update)
# turning_angle_slider.observe(update)
# flash_min_slider.observe(update)
# flash_max_slider.observe(update)
# trackon_checkbox.observe(update)

# setup_button.on_click(setup)
# go_button.on_click(run)

type(widgets)

## Main

In [ ]:
# Simulate a sky of flashing and moving fireflies
firefly_collection = FireflyCollection(num_males, step_size, num_total_steps, 
                                     flash_interval_min, flash_interval_max, 
                                     turning_angle_distribution,
                                     initial_arena_size)
all_fireflies_history = {}
for t_i in range(num_total_steps):
    for ff_i, ff in enumerate(firefly_collection):
        ff.pick_turning_angle()
        ff.check_flash()
        ff.take_step()
        hist_dict = ff.record_history() 
        
        all_fireflies_history[f"firefly_{ff_i}"] = hist_dict

In [ ]:
# Get min, max x, y across all fireflies to set arena 
x_min = num_total_steps * 2
x_max = - num_total_steps * 2
y_min = num_total_steps * 2
y_max = - num_total_steps * 2

for d in all_fireflies_history.values():
    x_min_d = min(d['trajectory_history'][:,0])
    x_max_d = max(d['trajectory_history'][:,0])
    y_min_d = min(d['trajectory_history'][:,1])
    y_max_d = max(d['trajectory_history'][:,1])
    
    if x_min_d < x_min:
        x_min = x_min_d
    if x_max_d > x_max:
        x_max = x_max_d
    if y_min_d < y_min:
        y_min = y_min_d
    if y_max_d > y_max:
        y_max = y_max_d
    
x_min, x_max, y_min, y_max

## Generate movie

In [ ]:
!rm MovieFrames/*.png

color = plt.cm.rainbow(np.linspace(0, 1, num_males))

track_on = True

for frame_i in range(num_total_steps):
    sys.stdout.write(f"\rFrame {frame_i+1}/{num_total_steps}")
    sys.stdout.flush()
    
    fig = plt.figure(figsize=(8,7))
    ax = fig.add_subplot(1, 1, 1)

    for ff_i, (key, val) in enumerate(all_fireflies_history.items()):
        ''' For a single firefly, extract its info and plot its trajectory. '''
        firefly_data = all_fireflies_history[key]

        x = firefly_data['trajectory_history'][:,0]
        y = firefly_data['trajectory_history'][:,1]
        flashes = np.where(firefly_data['flash_history'] == 1)
        x_flashes = x[flashes]
        y_flashes = y[flashes]
        
        # Plot position trajectory
        if track_on:
            plt.plot(x[0:frame_i], y[0:frame_i], lw=1, zorder=0, label=f"{ff_i+1}", c=color[ff_i])

        # Plot flashing patterns
        for global_i, f_i in enumerate(flashes[0]):
            if frame_i == f_i:
                plt.scatter(x_flashes[global_i], y_flashes[global_i], c='orange', s=40)
                
#             else:
#                 plt.scatter(0, 0, c="gray", s=40)

        # Set black background
        ax = plt.gca()
        ax.set_facecolor('xkcd:black')

        plt.xlim(x_min-10, x_max+10)
        plt.ylim(y_min-10, y_max+10)
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title(f'Firefly sky - time step: {frame_i+1}')
        
        if track_on:
            plt.legend(loc=(1.01, 0.2), ncol=2, title="Male firefly", columnspacing=0.5)
        
    plt.savefig(f'MovieFrames/t_{frame_i:03d}', bbox_inches='tight', dpi=200)
    plt.close()
        

In [ ]:
x_flashes[global_i]

In [ ]:
all_img_paths = np.sort(glob.glob("MovieFrames/*.png"))
all_imgs = np.array([cv2.imread(img) for img in all_img_paths])

def imgs2vid(imgs, outpath, fps=20):
    height, width, layers = imgs[0].shape
    fourcc = cv2.VideoWriter_fourcc("m", "p", "4", "v")
    video = cv2.VideoWriter(outpath, fourcc, fps, (width, height), True)
    
    for img in imgs:
        video.write(img)
        
    cv2.destroyAllWindows()
    video.release()
    
if track_on:
    imgs2vid(all_imgs, 'MovieFrames/FireflySky.mp4')
else:
    imgs2vid(all_imgs, 'MovieFrames/FireflySky_OnlyFlash.mp4')

Ideas for interface:
- Make it a story: Read in videos of track+flashing and only flashing
- Ask user to provide parameters from sliders
- Generate simulation
- Display results

In [ ]:
# widgets:
ticks = Text(description="Ticks:")
# sheep = Text(description="sheep:", margin=5, width="60px")
# wolves = Text(description="wolves:", margin=5, width="60px")
# grass = Text(description="grass/4:", margin=5, width="60px")
canvas = HTML(margin=10)
plot = HTML()
setup_button = Button(description="setup", width="47%", margin=5)
go_button = Button(description="go", width="47%", margin=5)

# Model parameters
num_males_slider = IntSlider(description="Number of male fireflies:", min=1, max=100, margin=5, width="100px")
step_size_slider = IntSlider(description="Step size:", min=1, max=5, margin=5, width="100px")
num_total_steps_slider = IntSlider(description="Number of total steps:", min=1, max=1000, margin=5, width="100px")
turning_angle_slider = IntSlider(description="Width of turning angle:", min=np.pi/10, max=np.pi/2, margin=5, width="100px")
flash_min_slider = IntSlider(description="Min flashing period:", min=1, max=100, margin=5, width="100px")
flash_max_slider = IntSlider(description="Max flashing period:", min=1, max=100, margin=5, width="100px")

# grass_checkbox = Checkbox(description="grass?", margin=5)
# grass_regrowth_time = IntSlider(description="grass_regrowth_time:",
#                                min=0, max=100, margin=5, width="300px")
# sheep_slider = IntSlider(description="initial_number_sheep:", min=0, max=250, margin=5, width="100px")
# wolves_slider = IntSlider(description="initial_number_wolves:", min=0, max=250, margin=5, width="100px")
# sheep_gain_from_food_slider = IntSlider(description="sheep_gain_from_food:", min=0, max=50, margin=5, width="100px")
# wolf_gain_from_food_slider = IntSlider(description="wolf_gain_from_food:", min=0, max=100, margin=5, width="100px")
# sheep_reproduce_slider = FloatSlider(description="sheep_reproduce:", min=0, max=20, margin=5, width="100px")
# wolf_reproduce_slider = FloatSlider(description="wolf_reproduce:", min=0, max=20, margin=5, width="100px")

# layout:
row1 = HBox([setup_button, go_button], background_color="lightblue", width="100%")
row2 = HBox([num_males_slider, step_size_slider], background_color="lightgreen", width="100%")
row3 = HBox([num_total_steps_slider, turning_angle_slider], background_color="lightgreen", width="100%")
row4 = HBox([flash_min_slider, flash_max_slider, background_color="lightgreen", width="100%")
row5 = HBox([plot], width="100%")
widgets = HBox([VBox([row1, row2, row3, row4, row5], width="60%"), 
                VBox([ticks, canvas], background_color="Khaki")], width="100%")

# row5 = HBox([sheep_reproduce_slider, wolf_reproduce_slider], background_color="lightgreen", width="100%")
# row61 = HBox([sheep], background_color="Khaki", width="100%")
# row62 = HBox([wolves], background_color="Khaki", width="100%")
# row63 = HBox([grass], background_color="Khaki", width="100%")

########################################
             
# update:
ticks.value = str(world.ticks)
canvas.value = str(world.draw(15))
plot.value = world.plot(6.0, 3.0)
grass_checkbox.value = world.use_grass
grass_regrowth_time.value = world.grass_regrowth_time
sheep_slider.value = world.initial_number_sheep
wolves_slider.value = world.initial_number_wolves
sheep_gain_from_food_slider.value = Sheep.GAIN_FROM_FOOD
wolf_gain_from_food_slider.value = Wolf.GAIN_FROM_FOOD
sheep_reproduce_slider.value = Sheep.REPRODUCE
wolf_reproduce_slider.value = Wolf.REPRODUCE
stats = world.get_stats()
sheep.value = str(stats[0])
wolves.value = str(stats[1])
grass.value = str(stats[2])
ticks.value = str(world.ticks)

def update(args):
    if args["name"] != "value":
        return
    owner = args["owner"]
    name = args["name"]
    value = args["new"]
    if owner == grass_checkbox:
        world.use_grass = value
    elif owner == grass_regrowth_time:
        world.grass_regrowth_time = value
    elif owner == sheep_slider:
        world.initial_number_sheep = value
    elif owner == wolves_slider:
        world.initial_number_wolves = value
    elif owner == sheep_gain_from_food_slider:
        Sheep.GAIN_FROM_FOOD = value
    elif owner == wolf_gain_from_food_slider:
        Wolf.GAIN_FROM_FOOD = value
    elif owner == sheep_reproduce_slider:
        Sheep.REPRODUCE = value
    elif owner == wolf_reproduce_slider:
        Wolf.REPRODUCE = value

def setup(widgets):
    world.setup()
    canvas.value = str(world.draw(15))
    plot.value = world.plot(6.0, 3.0)
    stats = world.get_stats()
    sheep.value = str(stats[0])
    wolves.value = str(stats[1])
    grass.value = str(stats[2])
    ticks.value = str(world.ticks)

def run(widgets):
    global thread
    if world.state == "stopped":
        go_button.description = "pause"
        w = {"canvas": canvas, "plot": plot, "ticks": ticks, 
             "sheep": sheep, "wolves": wolves, "grass": grass}
        thread = threading.Thread(target=lambda: world.run(w, 6.0, 3.0, 15))
        thread.start()
    else:
        go_button.description = "go"
        world.state = "stopped"
        thread.join()
        clear_output()
    
grass_checkbox.observe(update)
# FIX:
grass_regrowth_time.observe(update)
sheep_slider.observe(update)
wolves_slider.observe(update)
sheep_gain_from_food_slider.observe(update)
wolf_gain_from_food_slider.observe(update)
sheep_reproduce_slider.observe(update)
wolf_reproduce_slider.observe(update)
# END OF FIX

setup_button.on_click(setup)
go_button.on_click(run)

widgets

## Converting original MATLAB/R code with functional programming to Python

### 1. Male firefly correlated random walk

In [ ]:
# Parameters
# Based on these params, a firefly takes 1000 steps and flash every 100 steps (10x)
N = 1000           # Number of steps per trajectory
REALIZATIONS = 20  # Number of fireflies in arena 
v = 1.0            # Velocity aka step size
NreS = 100         # Number of steps bw flashes
sigma02 = np.pi/10 # Width of random walk turning angle distribution. Lower: straighter.

initial_arena_size = N

In [ ]:
# Correlated random walks
np.random.seed(42)

x = np.zeros((REALIZATIONS, N))
y = np.zeros((REALIZATIONS, N))
theta = np.zeros((REALIZATIONS, N))

# Pick random initial positions for all male FF (convert to - to + coordinates)
x[:,0] = initial_arena_size * (np.random.rand(x.shape[0]) - 0.5)
y[:,0] = initial_arena_size * (np.random.rand(y.shape[0]) - 0.5)
theta[:,0] = np.pi * (np.random.rand(theta.shape[0]) - 0.5)

# Loop over each male firefly
for realization_i in range(REALIZATIONS):
    
    # Each FF takes N steps total
    for step_i in range(1, N):
        
        # At each step, it chooses a random turning angle (radians)
        theta[realization_i, step_i] = theta[realization_i, step_i-1] + sigma02 * 2 * (np.random.rand()-0.5)
        
        # Update x and y position according to theta
        x[realization_i, step_i] = x[realization_i, step_i-1] + v * np.cos(theta[realization_i, step_i])
        y[realization_i, step_i] = y[realization_i, step_i-1] + v * np.sin(theta[realization_i, step_i])


In [ ]:
fig = plt.figure(figsize=(8,7))
ax = fig.add_subplot(1, 1, 1)

# Plot line tracks
plt.plot(x.T, y.T, lw=1, zorder=0, label="-")

# Plot flashing patterns
xx = x.T.copy()
yy = y.T.copy()
plt.scatter(xx[::NreS], yy[::NreS], c='orange', s=20)

plt.xlabel('X')
plt.ylabel('Y')

ax = plt.gca()
ax.set_facecolor('xkcd:black')

# plt.legend(loc=(1.01,.1), title=("Male firefly"))
plt.title('Firefly sky')
plt.show()

At first flashes, (assume synchronized flashing?) female picks a single male and attempts to track him until the end of simulation? At each time, firefly makes a guess on most likely male based on flash location, and build a history of that track

## Try to learn widgets

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

%matplotlib nbagg

Define the function that's changed each time a slider is changes

In [ ]:
def update_plot(amp, phase, freq):
    ax.clear()
    
    units = f"amp={amp} $(psi)$ \nphase={phase} $(secs)$ \nfreq={freq} $(Hz)$"
    y = amp * np.sin(2 * np.pi * (freq * x * phase))
    ax.plot(x, y, label=units)
    ax.legend(loc=1)
    ax.set_xlabel("$sec$")
    ax.set_ylabel("$psi$")
    ax.set_xlim()
    ax.set_ylim()
    plt.show()

In [ ]:
x = np.linspace(0, 2, 1000)
fig, ax = plt.subplots(1, figsize=(10, 4))
plt.suptitle('Sine wave')

# Create sliders
amp = widgets.FloatSlider(min=1, max=10, value=1, description="Amp: ")
phase = widgets.FloatSlider(min=0, max=5, value=0, description="Phase: ")
freq = widgets.FloatSlider(min=1, max=10, value=1, description="Freq: ")

# Link values of widgets to agruments for function
widgets.interactive(update_plot, amp=amp, phase=phase, freq=freq)

display(amp, phase, freq)